In [1]:
import ee
import geemap
import json
import geojson
from IPython.display import Image

import os
import pandas as pd
import requests
from geemap import geojson_to_ee, ee_to_geojson

In [2]:
#geemap.update_package()

In [3]:
Map = geemap.Map(center=[23.834063, 89.398476], zoom=7)
Map

Map(center=[23.834063, 89.398476], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

In [4]:
colorizedVis = {
  'min': 0.0,
  'max': 1.0,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],
}

In [5]:
ndvi_dataset = ee.ImageCollection('LANDSAT/LC08/C01/T1_8DAY_NDVI').filterDate('2017-01-01', '2017-12-31')
    
ndvi_colorized = ndvi_dataset.select('NDVI')

In [6]:
Map.addLayer(ndvi_colorized, colorizedVis, 'NDVI');

In [7]:
evi_dataset = ee.ImageCollection('LANDSAT/LC08/C01/T1_8DAY_EVI').filterDate('2017-01-01', '2017-12-31')
evi_colorized = evi_dataset.select('EVI')

In [8]:
Map.addLayer(evi_colorized, colorizedVis, 'EVI');

In [9]:
file_path = os.path.abspath('farmers_dropna_longlat.geojson')        

with open(file_path) as f:
    farmer_data = json.load(f)

In [10]:
for i in range(len(farmer_data['features'])):
    farmer_data['features'][i]['properties'] = {}

In [11]:
farmer_feature = geojson_to_ee(farmer_data)
Map.addLayer(farmer_feature, {}, "Farmers")

In [12]:
farmer_data.keys()

dict_keys(['type', 'features'])

In [125]:
all_points = []

for i in range(len(farmer_data['features'])):
    all_points.append(farmer_data['features'][i]['geometry']['coordinates'])

In [126]:
len(all_points)

686

In [127]:
all_points = all_points[:95]

In [136]:
all_points[90]

[29.32355, 25.93253]

In [128]:
poi = ee.Geometry.Point(all_points[1])
poi.getInfo()

{'type': 'Point', 'coordinates': [89.39408, 25.93322]}

In [129]:
poi = ee.Feature(ee.Geometry.Point(all_points[1]))
poi.getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [89.39408, 25.93322]},
 'properties': {}}

In [130]:
poi_fea = []

for i in range(len(all_points)):
    poi_fea.append(ee.Feature(ee.Geometry.Point(all_points[i])))

In [131]:
type(poi_fea[0])

ee.feature.Feature

In [132]:
roi = ee.FeatureCollection(poi_fea)

In [133]:
type(roi)

ee.featurecollection.FeatureCollection

In [134]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_dem_stats = os.path.join(out_dir, '95_points_ndvi_mean_2017_30.csv')  

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
geemap.zonal_statistics(ndvi_colorized, roi, out_dem_stats, statistics_type='MEAN', scale=30)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to C:\Users\kaies\Downloads\95_points_ndvi_mean_2017_30.csv


In [24]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_dem_stats = os.path.join(out_dir, 'evi_mean.csv')  

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
geemap.zonal_statistics(evi_colorized, roi, out_dem_stats, statistics_type='MEAN', scale=10000)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to C:\Users\kaies\Downloads\evi_mean.csv
